# Automation of TestVision - import/export of open questions

*This version: 15 June 2020*

Comments: h.datta@tilburguniversity.edu

**Requires Python 3.x**

**Preliminary and potentially unstable**

## Preamble
Loads a bunch of packages required to automate your browser. Run this before you go to the next cells.

In [1]:
# Load packages into memory
import urllib3
import datetime
from lxml import etree 
from selenium import webdriver
import time
import codecs
from lxml.cssselect import CSSSelector
from lxml.etree import fromstring
import re
import os

# Function to clean text from extra characters (e.g., new line characters, tabs)
def cleanstring(obj):
    return(re.sub(r"[\n\t\s]*", "", obj))

# Function to create a new directory if it does not exist yet
def makedir(dirname):
    try:
        os.stat(dirname)
    except:
        os.mkdir(dirname)
        
# Function to generate filename based on time of the computer
def timestamp_to_string(prefix='', extention='.html'):
    return(prefix+str(time.time()).replace('.','-')+extention)

def goto_1():
    content = browser.page_source
    tree = etree.fromstring(browser.page_source, parser=etree.HTMLParser())
    curcand=int(tree.xpath('//*[@id="vnu"]')[0].text)
    totcand=int(tree.xpath('//*[@id="vtot"]')[0].text)
    
    if curcand>1:
        steps_back=curcand-1
        while steps_back>0: 
            browser.find_element_by_css_selector('#vorige').click()
            steps_back=steps_back-1
            time.sleep(1.2)
            
def get_total():
    content = browser.page_source
    tree = etree.fromstring(browser.page_source, parser=etree.HTMLParser())
    totcand=int(tree.xpath('//*[@id="vtot"]')[0].text)
    return(totcand)

def get_question_names():
    questions = []
    el=browser.find_element_by_css_selector('#beoordelen_list')
    for row in el.find_elements_by_tag_name("tr"):
        fn=row.find_elements_by_tag_name('td')[0].text
        questions.append(fn.replace(':', '').replace('/',''))
    return(questions)

def open_question(name):
    cnt=0
    handl=browser.window_handles[0]
    browser.switch_to.window(handl)

    el=browser.find_element_by_css_selector('#beoordelen_list')
    for r in questions:
        if r==name: 
            el.find_elements_by_tag_name("tr")[cnt].find_elements_by_class_name('butwhite')[0].click()
        cnt+=1
    time.sleep(3)

def count_criteria():
    ncriteria = len(browser.find_elements_by_class_name('crit_tbl'))-2

    cnt=0
    qs=[]
    while (cnt<ncriteria):
        cnt+=1
        qs.append(cnt)
    print(qs)
    return(qs)


# INITIALIZE FILE
def init_header(filename):
    handl=browser.window_handles[1]
    browser.switch_to.window(handl)

    #filename = 'rct_quiz.txt'
    f=open(filename,'w',encoding='utf-8')
    f.write('tivo_session\ttivo_userid\ttivo_name\tanswer')

    qs = count_criteria() # array to indicate how many items to download per question #,2,3,4,5]
    
    for q in qs:
        f.write('\tq'+str(q)+'_currpoints\tq'+ str(q)+'_newpoints\tq'+str(q)+'_feedback')
    f.write('\n')
    f.close()
    
# DOWNLOAD ALL ANSWERS
def download_qs(filename):
    handl=browser.window_handles[1]
    browser.switch_to.window(handl)

    goto_1()

    totcand=get_total()
    f=open(filename,'a',encoding='utf-8')

    curr = 1

    while curr<=totcand:
        content = browser.page_source
        tree = etree.fromstring(browser.page_source, parser=etree.HTMLParser())

        # session name:
        session = browser.find_element_by_css_selector("#afnamenaam").text.replace('Grade per question','').replace('Beoordelen per vraag','')
        session=session.encode('ascii',errors='ignore').strip().decode("utf-8")

        # answer:
        answer = browser.find_element_by_css_selector("#antwoordvak").text.replace('\n',' | ').replace('\t','')

        # candidate:
        current_candidate_no = tree.xpath('//*[@id="vnu"]')[0].text
        current_candidate_name = tree.xpath('//*[@id="header_knaam"]')[0].text

        points = browser.find_elements_by_css_selector('#input_score > input')

        f.write(str(session)+'\t'+current_candidate_no+'\t'+current_candidate_name+'\t'+answer)

        for p in points:
            f.write('\t'+p.get_attribute("value")+'\t\t')

        f.write('\n')

        curr=curr+1
        browser.find_element_by_css_selector('#volgende').click()
        time.sleep(1.5)

    f.close()

# Manually log in to TestVision, please.

Run this code cell below to open your browser, and point you to Tilburg's login page.

In [20]:
# TestVision
browser=webdriver.Chrome()
browser.get('https://tilburgu.testvision.nl/online/ontwikkelaars/')

# log yourself and go to the question and select grade for a specific question

## Download all open questions


First, *point browser to the test, and the page "Afname - beoordelingsoverzicht"*.

Running the code below will download the answers to open questions for all questions and students, and save it to a .txt/.csv file.

In [123]:
handl=browser.window_handles[0]
browser.switch_to.window(handl)

questions = get_question_names()
print(questions)
questions[4:]
for q in questions: # [4:]:
    print('Getting ' + q)
    open_question(q)
    #init_header(q+'.txt')
    download_qs(q+'.txt')



['digital-market benefits and spotify business model', 'gordon_when_rct', 'mela_pitch', 'open - directory structure', 'open - discuss biasesendogeneity (gordon) - updrafts', 'rct fill in', 'tsh_backup', 'tsh_makefile_not_run', 'tsh_workflow_changes', 'webscraping_instagram']
Getting digital-market benefits and spotify business model
Getting gordon_when_rct
Getting mela_pitch
Getting open - directory structure
Getting open - discuss biasesendogeneity (gordon) - updrafts
Getting rct fill in
Getting tsh_backup
Getting tsh_makefile_not_run
Getting tsh_workflow_changes
Getting webscraping_instagram


# Upload points and feedback

For individual questions. Set fn to the file name of your graded sheets.

In [23]:
fn='import_when.csv'
import pandas
dat = pandas.read_csv(fn,sep='\t', encoding='utf-8')
dat

,tivo_session,tivo_userid,tivo_name,answer,q1_currpoints,q1_newpoints,q1_feedback
0,Quiz 3: Evaluating large scale field experiment,1,"Ackermans, R.A.J. (Rodney)",ITT = ATT when the share of compliers from the...,NaN,2,NaN
1,Quiz 3: Evaluating large scale field experiment,2,"Aert, J.A. van (Jeffrey)",The characteristics of the test group and the ...,NaN,2,NaN
2,Quiz 3: Evaluating large scale field experiment,3,"Arcangeli, L. (Lorenzo)","ln order to get equal ITT and ATT, the share o...",NaN,2,NaN
3,Quiz 3: Evaluating large scale field experiment,4,"Beelen, E. (Evi)",The intent-to-treat (ITT) is the average treat...,NaN,2,NaN
4,Quiz 3: Evaluating large scale field experiment,5,"Beldhuis, M. (Maaike)",The ITT is equal to the ATT when all users in ...,NaN,2,NaN
...,...,...,...,...,...,...,...
96,Quiz 3: Evaluating large scale field experiment,97,"Wel, K.M. van der (Kevin)",The chararetics of the observation from the Tr...,NaN,2,NaN
97,Quiz 3: Evaluating large scale field experiment,98,"Wiggers, S.B. (Stan)",100% of the people is exposed. So in the Expos...,NaN,2,NaN
98,Quiz 3: Evaluating large scale field experiment,40,"Kerkhof, Y.F.A. van de (Yarni)",The number of people that you want to treat sh...,NaN,0,NaN
99,Quiz 3: Evaluating large scale field experiment,80,"Smit, E. (Eeke)",The share of compliers (% of people that were ...,NaN,0,share of exposed users needs to be 100 percent...


In [24]:
import pandas
dat = pandas.read_csv(fn,sep='\t', encoding='utf-8')

handl=browser.window_handles[1]
browser.switch_to.window(handl)

goto_1()

curr = 1

npoints=count_criteria()
 
    
totcand=get_total()

while curr<=totcand:
    # answer:
    content = browser.page_source
    tree = etree.fromstring(browser.page_source, parser=etree.HTMLParser())
    
    current_candidate_no = tree.xpath('//*[@id="vnu"]')[0].text
    session = browser.find_element_by_css_selector("#afnamenaam").text.replace('Grade per question','').replace('Beoordelen per vraag','')
    session=session.encode('ascii',errors='ignore').strip().decode('utf-8')
    
    # Fill in points
    queried_data = dat.query('tivo_userid=='+current_candidate_no+'&tivo_session=="'+session+'"')
    
    antwoordvakjes = browser.find_elements_by_css_selector('#input_score > input')
    feedback_antwoordvakjes = browser.find_elements_by_css_selector('#feedback')

    for p in npoints:
          # f.write('\tq'+str(q)+'_currpoints\tq'+ str(q)+'_newpoints\tq'+str(q)+'_feedback')
#f
        puntjes=queried_data['q'+str(p)+'_newpoints']
        
        if (len(puntjes)>0):
            #antwoordvakjes = browser.find_element_by_css_selector('#input_score')
            antwoordvak=antwoordvakjes[p-1]
            antwoordvak.clear()
            try:
                pfloat = str(float(puntjes.item()))
            except:
                pfloat = str(float(puntjes.item().replace(',','.')))
                
            antwoordvak.send_keys(pfloat)
            print('points entered for ' + current_candidate_no)
            time.sleep(.5)
        else: 
            print('points not found for ' + current_candidate_no)

        # Fill in feedback
        try:
            feedback=queried_data['q'+str(p)+'_feedback'].item()
        except:
            feedback=''

        if ((str(feedback)!='nan') | len(str(feedback))>4):
            antwoordvak =feedback_antwoordvakjes[p-1]

            antwoordvak.clear()
            antwoordvak.send_keys(str(feedback))
            time.sleep(1)
    
    curr=curr+1
    browser.find_element_by_css_selector('#volgende').click()
    time.sleep(1)


[1]
points entered for 1
points entered for 2
points entered for 3
points entered for 4
points entered for 5
points entered for 6
points entered for 7
points entered for 8
points entered for 9
points entered for 10
points entered for 11
points entered for 12
points entered for 13
points entered for 14
points entered for 15
points entered for 16
points entered for 17
points entered for 18
points entered for 19
points entered for 20
points entered for 21
points entered for 22
points entered for 23
points entered for 24
points entered for 25
points entered for 26
points entered for 27
points entered for 28
points entered for 29
points entered for 30
points entered for 31
points entered for 32
points entered for 33
points entered for 34
points entered for 35
points entered for 36
points entered for 37
points entered for 38
points entered for 39
points entered for 40
points entered for 41
points entered for 42
points entered for 43
points entered for 44
points entered for 45
points entered 

# Inspect grading file that will be uploaded

In [ ]:
dat = pandas.read_csv('graded.txt',sep='\t')
dat

# Close session

In [ ]:
browser.close()